In [1]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

class smartCoarseGrainDataset(Dataset):
    def __init__(self, dataDir, transform=None):
        self.data = ImageFolder(dataDir, transform=transform)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        return x
    @property
    def classes(self):
        return self.data.classes
    


In [ ]:
currentTransforms = transforms.Compose([transforms.ToTensor()])
data = smartCoarseGrainDataset(dataDir = r"C:\Users\pgott\Desktop\Coding\Dataset\\", transform=currentTransforms)


dataLoader = DataLoader(data, batch_size=1, shuffle=True, num_workers=0)


In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, upscale_factor=2):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(32, 1 * (upscale_factor ** 2), (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.sigmoid(self.pixel_shuffle(self.conv3(x)))
        return x


In [5]:
upscaleModel = Net(upscale_factor=2)

for images, labels in dataLoader:
    break

forwardOut = upscaleModel(torch.mean(images, dim=1, keepdim=False))


In [9]:
print(forwardOut)
print(images.shape)

tensor([[[0.5126, 0.5122, 0.5051,  ..., 0.5066, 0.5043, 0.5016],
         [0.4924, 0.4994, 0.4959,  ..., 0.4964, 0.5125, 0.5010],
         [0.5140, 0.5086, 0.5035,  ..., 0.5077, 0.5045, 0.5031],
         ...,
         [0.4941, 0.5021, 0.4953,  ..., 0.5005, 0.5101, 0.5034],
         [0.5173, 0.5035, 0.5078,  ..., 0.5097, 0.5106, 0.5074],
         [0.5036, 0.5075, 0.5020,  ..., 0.5035, 0.5033, 0.5040]]],
       grad_fn=<SigmoidBackward0>)
torch.Size([1, 3, 1010, 1010])


In [13]:
print(labels)

tensor([1])


training loop

In [ ]:
loss = nn.MSELoss()
optimizer = optim.Adam(upscaleModel.parameters(), lr=0.001)

train_loss, val_loss = [], []
num_epoch = 10


# Pick GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move your model to the device
upscaleModel = upscaleModel.to(device)


for epoch in range(num_epoch):
    upscaleModel.train()

    for images, labels in dataLoader:
        optimizer.zero_grad()
        forwardOut = upscaleModel(images.to(device))

        MSE = loss(forwardOut, images.to(device))

c:\Users\pgott\miniconda3\envs\coarseGrain\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 3, 1010, 1010])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.9966)